Comparing several methods for creating out of sample predictive intervals

Methods:
- Quantile of residuals
- Quantile regression
- Jacknife+
- Bayesian regression


In [1]:
%load_ext autoreload
%autoreload 2

### Import python modules

In [2]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from copy import deepcopy as dcp

sys.path.append("..")

### Import predictive interval modules

In [3]:
# Import data simulator
from predictive_intervals.data_simulator import DataSimulator

# Import models
from predictive_intervals.models.predictive_intervals_model import PredictiveIntervalModel
from predictive_intervals.models.linear_regression import LinearModel
from predictive_intervals.models.quantile_regression import QuantileRegression
from predictive_intervals.models.jacknife_plus_regression import JacknifePlus
from predictive_intervals.models.bayesian_regression import BayesianRegression

Define data simulator parameters

In [4]:
# Number of data points to sample
n_points = 10000
pct_train = 0.5

# Model parameters
sigma = 3.
alpha = 1.
beta = 2.

Simulate data

In [5]:
data_sim = DataSimulator.generate_lm_data(n_points=n_points,
                                          pct_train=pct_train,
                                          alpha=alpha,
                                          beta=beta,
                                          sigma=np.sqrt(sigma))

# Run Experiments

Run few repeated experiments, time them and collects statistics

In [21]:
n_experiments = 10 # or as Andrew Gelman would say, in statistics, 30 = infinity

Set desired level of the predictive interval and instantiate all models

In [22]:
alpha = 0.1

In [29]:
linear_regression = LinearModel(alpha=alpha)
quantile_regression = QuantileRegression(alpha=alpha)
jacknife_plus_regression = JacknifePlus(alpha=alpha)
bayesian_regression = BayesianRegression(alpha=alpha)

methods = [linear_regression, quantile_regression, jacknife_plus_regression, bayesian_regression]

In [30]:
def run_experiment(method: PredictiveIntervalModel,
                   data_simulator: DataSimulator,
                   n_experiments: int = 1):
    
    # Declare lists for storing results
    run_times = []
    hit_ratios = []
    interval_lengths = []
    
    # Loop over number of experiments, measure time and collect statistics
    for i in range(n_experiments):
        
        # Generate new data
        data_sim = data_simulator.generate_lm_data(n_points=n_points,
                                          pct_train=pct_train,
                                          alpha=alpha,
                                          beta=beta,
                                          sigma=np.sqrt(sigma))
        
        # Single run of an experiment
        start_time = time.time()
        method.get_predictive_intervals(data=data_sim)
        end_time = time.time()
        
        # Append results
        run_times.append(end_time - start_time)
        hit_ratios.append(method.hit_ratio)
        interval_lengths.append(method.avg_length)
        
    # Put all results into dataframe
    results = pd.DataFrame(list(zip(run_times, hit_ratios, interval_lengths)), columns=['time', 'hit_ratio', 'int_length'])
    
    return results

In [31]:
res = {}
for method in methods:
    method_results = run_experiment(method=method,
                                    data_simulator = DataSimulator,
                                    n_experiments=n_experiments)
    res[method] = method_results

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_c44f9f88c1db53571cc32dc48839b9d6 NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_c44f9f88c1db53571cc32dc48839b9d6 NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_c44f9f88c1db53571cc32dc48839b9d6 NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_c44f9f88c1db53571cc32dc48839b9d6 NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_c44f9f88c1db53571cc32dc48839b9d6 NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_c44f9f88c1db53571cc32dc48839b9d6 NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_c44f9f88

In [33]:
res[methods[0]]

,time,hit_ratio,int_length
0,0.001500,0.9,5.797185
1,0.000354,0.9,5.677511
2,0.000338,0.9,5.683132
3,0.000357,0.9,5.619180
4,0.000851,0.9,5.764408
5,0.000398,0.9,5.669571
6,0.000307,0.9,5.695586
7,0.000350,0.9,5.637588
8,0.000327,0.9,5.880803
9,0.000309,0.9,5.623381


In [34]:
res[methods[1]]

,time,hit_ratio,int_length
0,0.146294,0.8974,5.731846
1,0.000113,0.9038,5.731926
2,0.000188,0.9018,5.732212
3,0.000192,0.9078,5.731660
4,0.000127,0.9090,5.733295
5,0.000152,0.9008,5.732740
6,0.000112,0.9100,5.731011
7,0.000111,0.8976,5.731631
8,0.000114,0.9034,5.733274
9,0.000186,0.9062,5.731343


In [35]:
res[methods[2]]

,time,hit_ratio,int_length
0,393.455976,0.8964,5.661246
1,396.066128,0.8890,5.616014
2,397.282521,0.9150,5.830604
3,393.890152,0.9102,5.802167
4,394.288381,0.9016,5.808514
5,393.507593,0.9010,5.726242
6,392.926764,0.8960,5.657998
7,392.259675,0.8984,5.668577
8,393.840067,0.9024,5.617148
9,394.269000,0.8996,5.594979


In [36]:
res[methods[3]]

,time,hit_ratio,int_length
0,147.120050,0.8972,5.657509
1,144.026030,0.9024,5.700724
2,142.912685,0.9002,5.720997
3,143.311437,0.8910,5.616657
4,144.186823,0.8988,5.734024
5,146.441094,0.8916,5.652766
6,139.966973,0.9024,5.767894
7,142.330303,0.8936,5.701947
8,142.341050,0.8994,5.714071
9,144.706161,0.8938,5.666935
